In [1]:
from kiteconnect import KiteConnect

In [2]:
import logging
import requests

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [9]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [10]:
# Get the Request Token from the redirect URL

request_token = "WbhIjx7WlQxJwC1ir3Px1qqb44bDM2du"

# Get Access Token

In [11]:
data = kite.generate_session(request_token, api_secret=api_secret)
access_token = data["access_token"]
kite.set_access_token(access_token)
print(access_token)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


i69P4YjMJZ50wxtnAjUfwydHfc8q0Xvi


In [12]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BSE', 'BFO', 'CDS', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# Get Instrument Tokens

In [13]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [14]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

# Get Exponential Moving Average

In [15]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

# Stochastic Momentum Indicator

In [16]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# Zerodha Testing LIVE Deployment TradeLog

In [17]:
def place_trade(trade_log, current_time, current_price, current_short_term_mavg, current_long_term_mavg, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_short_term_mavg, current_long_term_mavg, current_transaction_type, current_status]
    print(current_time, current_transaction_type, current_price, current_status)
    return trade_log, current_transaction_type, current_status

# Pre-Flight Checks

### WARNING: Don't forget to update the date and time in 'deply_till' variable

In [23]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '3minute'
short_term_emavg_duration = 4
long_term_emavg_duration = 20

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=9, day=3, hour=15 , minute=30 , second=0)

# first transaction time
next_transaction_time = datetime(2020, 9, 4, 9, 53, 58)
candle = 3

# THE PAYLOAD

In [22]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 863438


In [24]:
price_log = pd.DataFrame(columns=['time','price',str(short_term_emavg_duration)+'d_EMAvg',str(long_term_emavg_duration)+'d_EMAvg'])
trade_log = pd.DataFrame(columns=['time','price',str(short_term_emavg_duration)+'d_EMAvg',str(long_term_emavg_duration)+'d_EMAvg','transaction_type','status'])
current_status='closed'
while datetime.strftime(datetime.now(), "%H:%M:%S")!=datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M:%S"):
    sleep(1)
    test_end = datetime.now()
    test_start = test_end - timedelta(days=1)
    print(test_end)
    if datetime.strftime(test_end, "%H:%M:%S")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M:%S"):
        for _ in range(5):
            try:
                hist_data = get_historical_data(instoken, test_start, test_end, interval=candle_size)
                break
            except requests.exceptions.ReadTimeout as e:
                logging.exception("Timeout Error - while getting ltp: \n %s", e)
                logging.info("Retrying")
                continue
        emav_df = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'price':list(hist_data['close'])}).set_index('date', drop=True)
        for window in [short_term_emavg_duration, long_term_emavg_duration]:
            emav_df = get_exponential_moving_avg(emav_df, window)
        current_time = list(emav_df.index)[-1]
        current_price = emav_df.loc[current_time, 'price']
        current_short_term_mavg = emav_df.loc[current_time, str(short_term_emavg_duration)+'d_EMAvg']
        current_long_term_mavg = emav_df.loc[current_time, str(long_term_emavg_duration)+'d_EMAvg']
        price_log.loc[len(price_log)]=[current_time, current_price, current_short_term_mavg, current_long_term_mavg]
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
                break
        else:
            break
    
    if test_end.hour==next_transaction_time.hour and test_end.minute==next_transaction_time.minute and test_end.second>=next_transaction_time.second:
        for _ in range(5):
            try:
                hist_data = get_historical_data(instoken, test_start, test_end, interval=candle_size)
                break
            except requests.exceptions.ReadTimeout as e:
                logging.exception("Timeout Error - while getting ltp: \n %s", e)
                logging.info("Retrying")
                continue
        
        emav_df = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'price':list(hist_data['close'])}).set_index('date', drop=True)
        for window in [short_term_emavg_duration, long_term_emavg_duration]:
            emav_df = get_exponential_moving_avg(emav_df, window)
        current_time = datetime.strftime(next_transaction_time-timedelta(minutes=candle-1, seconds=58), "%Y-%m-%d %H:%M:%S")
        current_price = emav_df.loc[current_time, 'price']
        current_short_term_mavg = emav_df.loc[current_time, str(short_term_emavg_duration)+'d_EMAvg']
        current_long_term_mavg = emav_df.loc[current_time, str(long_term_emavg_duration)+'d_EMAvg']
        price_log.loc[len(price_log)]=[current_time, current_price, current_short_term_mavg, current_long_term_mavg]
        
        if current_short_term_mavg>current_long_term_mavg:
            transaction_type='buy'
            if current_status=='closed':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
            elif current_status=='open' and current_transaction_type=='sell':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
        elif current_short_term_mavg<current_long_term_mavg:
            transaction_type='sell'
            if current_status=='closed':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
            elif current_status=='open' and current_transaction_type=='buy':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
        
        next_transaction_time = next_transaction_time+timedelta(minutes=candle)

2020-09-04 09:51:52.841155
2020-09-04 09:51:53.841233
2020-09-04 09:51:54.841278
2020-09-04 09:51:55.841412
2020-09-04 09:51:56.841485
2020-09-04 09:51:57.841529
2020-09-04 09:51:58.842264
2020-09-04 09:51:59.842340
2020-09-04 09:52:00.842439
2020-09-04 09:52:01.844194
2020-09-04 09:52:02.845038
2020-09-04 09:52:03.845107
2020-09-04 09:52:04.845730
2020-09-04 09:52:05.846887
2020-09-04 09:52:06.848041
2020-09-04 09:52:07.848043
2020-09-04 09:52:08.849116
2020-09-04 09:52:09.849193
2020-09-04 09:52:10.849269
2020-09-04 09:52:11.850417
2020-09-04 09:52:12.850550
2020-09-04 09:52:13.850620
2020-09-04 09:52:14.850694
2020-09-04 09:52:15.850824
2020-09-04 09:52:16.851741
2020-09-04 09:52:17.853165
2020-09-04 09:52:18.853230
2020-09-04 09:52:19.855139
2020-09-04 09:52:20.855697
2020-09-04 09:52:21.857098
2020-09-04 09:52:22.858171
2020-09-04 09:52:23.858301
2020-09-04 09:52:24.858378
2020-09-04 09:52:25.859463
2020-09-04 09:52:26.859542
2020-09-04 09:52:27.860030
2020-09-04 09:52:28.860330
2

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 09:53:58.929701


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+09%3A53%3A58&to=2020-09-04+09%3A53%3A58&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2545


2020-09-04 09:51:00 sell 23186.95 open
2020-09-04 09:54:00.680029
2020-09-04 09:54:01.681991
2020-09-04 09:54:02.683156
2020-09-04 09:54:03.683234
2020-09-04 09:54:04.684570
2020-09-04 09:54:05.686649
2020-09-04 09:54:06.687735
2020-09-04 09:54:07.687812
2020-09-04 09:54:08.687888
2020-09-04 09:54:09.688977
2020-09-04 09:54:10.690936
2020-09-04 09:54:11.691671
2020-09-04 09:54:12.692773
2020-09-04 09:54:13.693867
2020-09-04 09:54:14.695072
2020-09-04 09:54:15.695279
2020-09-04 09:54:16.695954
2020-09-04 09:54:17.698337
2020-09-04 09:54:18.699430
2020-09-04 09:54:19.700536
2020-09-04 09:54:20.701629
2020-09-04 09:54:21.702995
2020-09-04 09:54:22.704497
2020-09-04 09:54:23.704948
2020-09-04 09:54:24.705215
2020-09-04 09:54:25.706363
2020-09-04 09:54:26.707514
2020-09-04 09:54:27.708612
2020-09-04 09:54:28.709709
2020-09-04 09:54:29.710958
2020-09-04 09:54:30.712663
2020-09-04 09:54:31.712786
2020-09-04 09:54:32.713778
2020-09-04 09:54:33.713887
2020-09-04 09:54:34.713963
2020-09-04 09:54

...

2020-09-04 15:27:00.316335
2020-09-04 15:27:01.317758
2020-09-04 15:27:02.319202
2020-09-04 15:27:03.319605
2020-09-04 15:27:04.321028
2020-09-04 15:27:05.322452
2020-09-04 15:27:06.323879
2020-09-04 15:27:07.324299
2020-09-04 15:27:08.325723
2020-09-04 15:27:09.327167
2020-09-04 15:27:10.327570
2020-09-04 15:27:11.328996
2020-09-04 15:27:12.330418
2020-09-04 15:27:13.331840
2020-09-04 15:27:14.333264
2020-09-04 15:27:15.334688
2020-09-04 15:27:16.336112
2020-09-04 15:27:17.337555
2020-09-04 15:27:18.337960
2020-09-04 15:27:19.339383
2020-09-04 15:27:20.340806
2020-09-04 15:27:21.342229
2020-09-04 15:27:22.343673
2020-09-04 15:27:23.344076
2020-09-04 15:27:24.345500
2020-09-04 15:27:25.346923
2020-09-04 15:27:26.348347
2020-09-04 15:27:27.349770
2020-09-04 15:27:28.351193
2020-09-04 15:27:29.352619
2020-09-04 15:27:30.354040
2020-09-04 15:27:31.355463
2020-09-04 15:27:32.357383
2020-09-04 15:27:33.358803
2020-09-04 15:27:34.360247
2020-09-04 15:27:35.361650
2020-09-04 15:27:36.363073
2

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 15:28:00.397236


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+15%3A28%3A00&to=2020-09-04+15%3A28%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2611


2020-09-04 15:27:00 buy 23072.45 closed


In [25]:
trade_log.to_excel('LIVE Deployment Logs\RUN1A-HD-20200904-3min_candle_prev_candle_close_4vs20_emavg.xlsx')